In [90]:
import pandas as pd
import math

In [109]:
#train_data_m = pd.read_csv("train/gielda.txt", header=None)
#train_data_m = pd.read_csv("train/car_data.txt", header=None)
train_data_m = pd.read_csv("train/car.data", header=None)
train_data_m

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [101]:
def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0]
    total_entr = 0
    for c in class_list: #dla kazdej klasy w kolumnie
        total_class_count = train_data[train_data[label] == c].shape[0] # liczba klas
        total_class_entr = - (total_class_count/total_row)*math.log2(total_class_count/total_row) # entropia klasy
        total_entr += total_class_entr # dodawanie entropii z klasy do calkowitej sumy entropi wszyskich klas
        print(total_entr)
    return total_entr

In [100]:
calc_total_entropy(train_data_m, 6, train_data_m[6].unique())

1.2057409700121753

In [93]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] # zliczanie wierszy klasy c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #praństwo klasy
            entropy_class = - probability_class * math.log2(probability_class)  #obliczanie entropii
        entropy += entropy_class
    return entropy

In [94]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() # unikatowe wartości zmiennej
    total_row = train_data.shape[0]
    feature_info = 0.0
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] # filtrowanie wierszy za pomocą zmiennej
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy # obliczanie info
    return calc_total_entropy(train_data, label, class_list) - feature_info #obliczanie gain info

In [95]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label)
    max_info_gain = -1
    max_info_feature = None
    for feature in feature_list:
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: # wybieranie zmiennej z najwieksza wartoscia gain info
            max_info_gain = feature_info_gain
            max_info_feature = feature
    return max_info_feature

In [96]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #slownik z liczba zliczonych unikalnych wartosci zmiennej
    tree = {} 
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value]        
        assigned_to_node = False 
        for c in class_list: 
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] 
            if class_count == count:
                tree[feature_value] = c 
                train_data = train_data[train_data[feature_name] != feature_value] 
                assigned_to_node = True
        if not assigned_to_node: 
            tree[feature_value] = "?"   
    return tree, train_data

In [97]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: # jesli dane sa puste po aktualizacji (w petli)
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #znalezienie zmiennej z najwieksza iloscia informacji (info gain)
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) 
        next_root = None        
        if prev_feature_value != None: # dodaj do posredniego wezla drzewa
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #dodaj 
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): # interacja wezla drzewa
            if branch == "?": #jesli jest rozszerzalny
                feature_value_data = train_data[train_data[max_info_feature] == node] #przy użyciu zaktualizowanego zbioru danych
                make_tree(next_root, node, feature_value_data, label, class_list) # wywolanie funkcji rekurencyjnej

In [98]:
def id3(train_data_m, label):
    train_data = train_data_m.copy()
    tree = {} 
    class_list = train_data[label].unique() # uzyskiwanie unikalnych klas etykiety
    make_tree(tree, None, train_data_m, label, class_list) # wywolanie funkcji rekurencyjnej
    return tree

In [111]:
tree = id3(train_data_m, 6)

class Wezel:
    potomkowie = None
    atrybut = None
    wartosc = None
    klasa = None

def wyswietlDrzewo(drzewo,przesuniecie):
    if drzewo.atrybut!=None:
        print(" "*przesuniecie,end="")
        if drzewo.wartosc: print(drzewo.wartosc,end=" -> ")
        print("Atrybut",drzewo.atrybut)
        for p in drzewo.potomkowie:
            wyswietlDrzewo(p,przesuniecie+4)
    else:
        print(" "*przesuniecie,end="")
        print(drzewo.wartosc, "->" ,drzewo.klasa)

def buduj(test):
    w = Wezel()
    w.potomkowie = []
    klucz, element = test.popitem()
    w.atrybut = klucz
    for t in element:
        if isinstance(element[t], (dict)):
            nowyWezel=buduj(element[t])
            nowyWezel.wartosc = t
            w.potomkowie.append(nowyWezel)
        else:
            nowyWezel = Wezel()
            nowyWezel.wartosc = t
            nowyWezel.klasa = element[t]
            w.potomkowie.append(nowyWezel)
    return w

#test = {1:{"n":"tak", "m":{2:{"a":"nie", "b":"tak"}}, "o":"nie"}} #testowe dane
#test = {1: {'old': 'down', 'mid': {2: {'yes': 'down', 'no': 'up'}}, 'new': 'up'}}
#w = buduj(test)
w = buduj(tree)
w
wyswietlDrzewo(w, 0)

0.3599863227441141
0.8421918786201835
1.0202091122971373
1.2057409700121753
0.3599863227441141
0.8421918786201835
1.0202091122971373
1.2057409700121753
0.3599863227441141
0.8421918786201835
1.0202091122971373
1.2057409700121753
0.3599863227441141
0.8421918786201835
1.0202091122971373
1.2057409700121753
0.3599863227441141
0.8421918786201835
1.0202091122971373
1.2057409700121753
0.3599863227441141
0.8421918786201835
1.0202091122971373
1.2057409700121753
0.4277459570425642
0.9521434273902636


ValueError: math domain error

In [125]:
train_data_m[6][300:350]

300    unacc
301    unacc
302    unacc
303    unacc
304    unacc
305    unacc
306    unacc
307    unacc
308      acc
309    unacc
310      acc
311      acc
312    unacc
313      acc
314      acc
315    unacc
316    unacc
317      acc
318    unacc
319      acc
320      acc
321    unacc
322      acc
323      acc
324    unacc
325    unacc
326    unacc
327    unacc
328    unacc
329    unacc
330    unacc
331    unacc
332    unacc
333    unacc
334    unacc
335      acc
336    unacc
337    unacc
338      acc
339    unacc
340      acc
341      acc
342    unacc
343    unacc
344    unacc
345    unacc
346    unacc
347      acc
348    unacc
349      acc
Name: 6, dtype: object